In [2]:
from datetime import datetime, timedelta
from time import sleep
import os
import re
import requests
import wget

In [3]:
from pprint import pprint
a = {"name":"\"Wald der Zukunft\" Stadtbergen","exposure":"outdoor","sensors":[{"title":"Temperatur","unit":"°C","sensorType":"HDC1080","icon":"osem-thermometer","_id":"617ab96c8b4e03001b05b493","lastMeasurement":"632fb60751771e001b14ad80"},{"title":"rel. Luftfeuchte","unit":"%","sensorType":"HDC1080","icon":"osem-humidity","_id":"617ab96c8b4e03001b05b492","lastMeasurement":"632fb60751771e001b14ad81"},{"_id":"617ab96c8b4e03001b05b491","title":"Luftdruck","unit":"hPa (=mBar)","sensorType":"BMP280","icon":"osem-barometer","lastMeasurement":"632fb60751771e001b14ad7f"},{"title":"Beleuchtungsstärke","unit":"lx","sensorType":"TSL45315","icon":"osem-brightness","_id":"617ab96c8b4e03001b05b490","lastMeasurement":"632fb60751771e001b14ad7e"},{"title":"UV-Intensität","unit":"μW/cm²","sensorType":"VEML6070","icon":"osem-brightness","_id":"617ab96c8b4e03001b05b48f","lastMeasurement":"632fb60751771e001b14ad7d"},{"title":"PM10","unit":"µg/m³","sensorType":"SDS 011","icon":"osem-cloud","_id":"617ab96c8b4e03001b05b48e","lastMeasurement":"632fb60751771e001b14ad82"},{"title":"PM2.5","unit":"µg/m³","sensorType":"SDS 011","icon":"osem-cloud","_id":"617ab96c8b4e03001b05b48d","lastMeasurement":"632fb60751771e001b14ad83"},{"_id":"617ab96c8b4e03001b05b48c","title":"Bodenfeuchte","unit":"%","sensorType":"SMT50","icon":"osem-humidity","lastMeasurement":"632fb60751771e001b14ad84"},{"title":"Bodentemperatur","unit":"°C","sensorType":"SMT50","icon":"osem-thermometer","_id":"617ab96c8b4e03001b05b48b","lastMeasurement":"632fb60751771e001b14ad85"}],"model":"homeV2WifiFeinstaub","id":"617ab96c8b4e03001b05b489"}
pprint(a['sensors'])

[{'_id': '617ab96c8b4e03001b05b493',
  'icon': 'osem-thermometer',
  'lastMeasurement': '632fb60751771e001b14ad80',
  'sensorType': 'HDC1080',
  'title': 'Temperatur',
  'unit': '°C'},
 {'_id': '617ab96c8b4e03001b05b492',
  'icon': 'osem-humidity',
  'lastMeasurement': '632fb60751771e001b14ad81',
  'sensorType': 'HDC1080',
  'title': 'rel. Luftfeuchte',
  'unit': '%'},
 {'_id': '617ab96c8b4e03001b05b491',
  'icon': 'osem-barometer',
  'lastMeasurement': '632fb60751771e001b14ad7f',
  'sensorType': 'BMP280',
  'title': 'Luftdruck',
  'unit': 'hPa (=mBar)'},
 {'_id': '617ab96c8b4e03001b05b490',
  'icon': 'osem-brightness',
  'lastMeasurement': '632fb60751771e001b14ad7e',
  'sensorType': 'TSL45315',
  'title': 'Beleuchtungsstärke',
  'unit': 'lx'},
 {'_id': '617ab96c8b4e03001b05b48f',
  'icon': 'osem-brightness',
  'lastMeasurement': '632fb60751771e001b14ad7d',
  'sensorType': 'VEML6070',
  'title': 'UV-Intensität',
  'unit': 'μW/cm²'},
 {'_id': '617ab96c8b4e03001b05b48e',
  'icon': 'osem-

In [4]:
BASE_DIR = "./Data/"
sense_boxes = [
    "617ab96c8b4e03001b05b489",
    "633315b01ad7df001b349a76",
    "601307eb571de9001b1b6c6c"
]

In [6]:
def get_urls(string):
    urls = []
    for sense_box_id in sense_boxes:
        big_string = string
        value = sense_box_id
        pattern = rf'<a href="(\.\/{value}[^"]*)">'
        match = re.search(pattern, big_string)

        if match:
            urls.append(match.group(1).strip(".").rstrip("/"))
    
    return urls

def get_assignment(assignment_dict):
    sensor_names = assignment_dict.get("sensors")
    
    assignment = dict()

    for sensor in sensor_names:
        assignment[sensor.get('_id')] = {'Title': sensor.get('title'), 'Unit': sensor.get('unit')}
    return assignment

def get_files(string):
    # <a href="./617ab96c8b4e03001b05b48b-2022-09-24.csv">
    pattern = r'href=["\']([^"\']*?\.(?:csv|json))["\']'

    # Find all matches
    matches = [match.strip('.') for match in re.findall(pattern, string)]
    return matches

# Each URL
# Download each file and the json file

In [7]:
start_date = datetime(2023, 9, 28)
base_url = 'https://archive.opensensemap.org/'
day_count = 800

for single_date in (start_date + timedelta(n) for n in range(day_count)):
    if single_date.date() > datetime.today().date():
        print(single_date, "stopped")
        break

    date = single_date.strftime(format="%Y-%m-%d")
    date_folder = BASE_DIR + date

    # Check if date has already been scraped
    if os.path.exists(date_folder):
        continue

    # Create the folder
    current_directory = os.getcwd()
    final_directory = os.path.join(current_directory, date_folder)
    os.makedirs(date_folder)

    # Get all the Senseboxes sub URLs
    urls = get_urls(requests.get(f'https://archive.opensensemap.org/{date}/').text)

    for url in urls:
        url_folder = date_folder + "/" + url
        os.makedirs(url_folder)        
        
        # Wget all the files
        # download all the files
        for file in get_files(requests.get(f'https://archive.opensensemap.org/{date}{url}').text):
            file = f'https://archive.opensensemap.org/{date}{url}{file}'
            out_dir = date_folder + url
            wget.download(file, out=out_dir, bar=None)
        # file = "https://archive.opensensemap.org/2022-09-24/617ab96c8b4e03001b05b489-_Wald_der_Zukunft__Stadtbergen/617ab96c8b4e03001b05b48b-2022-09-24.csv"
        # wget.download(file, out=date_folder+"/617ab96c8b4e03001b05b489-_Wald_der_Zukunft__Stadtbergen/", bar=None)
        assignment_file = a  # read the downloaded json file
        assignments = get_assignment(assignment_file)

        # Load the files into a table for every station and every day solo
        #   so you can merge all of them at once instead of step wise.
    
    print(single_date)
    sleep(1.0)

2023-12-06 00:00:00
2024-06-22 00:00:00
2024-06-23 00:00:00
2024-06-24 00:00:00
2024-06-25 00:00:00
2024-06-26 00:00:00
2024-06-27 00:00:00
2024-06-28 00:00:00 stopped
